In [ ]:
import numpy as np
import pandas as pd


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pip install rank-bm25


In [15]:
import pandas as pd
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Load your dataset
df = pd.read_csv("/content/drive/MyDrive/total1.csv")

# Combining title, ingredients, and steps into a single text for each recipe
df['combined_text'] = df['Recipe Title'] + " " + df['Ingredients'].astype(str) + " " + df['Steps'].astype(str)

# Tokenization
tokenized_data = [doc.split(" ") for doc in df['combined_text']]

# Creating the BM25 object
bm25 = BM25Okapi(tokenized_data)

def search_recipes(query):
    query = query.split(" ")
    scores = bm25.get_scores(query)
    top_indexes = np.argsort(scores)[::-1][:10]  # Get top 10 indexes
    return df.iloc[top_indexes]

# Example query
search_results = search_recipes("chocolate cake")
print(search_results[['Recipe Title', 'Ingredients', 'Steps']])

                                            Recipe Title  \
6064                         Triple-Chocolate Layer Cake   
8528                              Chocolate Biscuit Cake   
41245                                Halloween cake pops   
23653                  Candy-Filled Chocolate Easter Egg   
42985                        Christmas pudding cake pops   
8346              Double-Chocolate Tangerine Mousse Cake   
7283                         Chocolate-Mint Brownie Cake   
45736                               Czech chocolate cake   
3729   Peanut Butter Cake with Chocolate-Peanut Butte...   
27670                                     Guacamole Cake   

                                             Ingredients  \
6064   ['Serves 8 to 10', 'For cake', '1/2 cup cold b...   
8528   ['Makes 8 servings', '1/2 teaspoon butter, for...   
41245  ['100g/4oz madeira cake', '100g chocolate sand...   
23653  ['One 9-by-1-inch confetti cake round', '1 pou...   
42985  ['200g madeira cake', '140g-160g

In [5]:
import pandas as pd
from rank_bm25 import BM25Okapi
import numpy as np

# Load your dataset
# df = pd.read_csv('your_dataset.csv')

# Preprocessing: assuming 'Ingredients' is a list of ingredients in string format
df['cleaned_ingredients'] = df['Ingredients'].apply(lambda x: x.lower().split(','))

# Tokenize: split each ingredient list into words
tokenized_ingredients = [doc for doc in df['cleaned_ingredients']]

# Create the BM25 object
bm25 = BM25Okapi(tokenized_ingredients)

def search_recipes_by_ingredients(ingredients):
    query = ingredients.lower().split(',')
    scores = bm25.get_scores(query)
    top_indexes = np.argsort(scores)[::-1][:10]  # Adjust number of results as needed
    return df.iloc[top_indexes]

# Example query: search for recipes with specific ingredients
search_results = search_recipes_by_ingredients("flour, sugar, eggs, cream cheese")
print(search_results[['Recipe Title', 'Ingredients', 'Steps']])


                                            Recipe Title  \
13879                            Citrus-and-Dill Gravlax   
45787                                      Keto brownies   
15259                Zucchini Stuffed with Feta and Dill   
15267                              Japanese Cold Noodles   
15266                   Braised Shrimp and Creamy Endive   
15265        Fresh Orange Slices with Honey and Cinnamon   
15264  Turkey and Sweet-Potato Croquettes with Cranbe...   
15263               Chocolate-Orange Easter-Egg Truffles   
15262                               Chocolate Cheesecake   
15261                              Gruyère Walnut Wafers   

                                             Ingredients  \
13879  ['12 servings', '1 bunch dill, plus sprigs for...   
45787  ['120g nut butter (we used almond butter)', '1...   
15259  ['Serves 4', 'four 1/2-pound zucchini, scrubbe...   
15267  ['Makes 6 servings', 'For dipping sauce', '2 c...   
15266  ['Makes 2 servings', '1 lb Belgi

In [4]:
# for GPU use
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ctransformers: filename=ctransformers-0.2.27-cp310-cp310-linux_x86_64.whl size=2338851 sha256=62a7d2d0b35d5f1b617ee3dbcd931b98ad3f55770fbdad2e0bb013d3518e9f3e
  Stored in directory: /root/.cache/pip/wheels/dd/54/e9/32364da8eee84a2b0b412394983c15add18816c507e90f02d8
Successfully built ctransformers


In [1]:
import pandas as pd
from rank_bm25 import BM25Okapi
import numpy as np
from ctransformers import AutoModelForCausalLM

# Assume df is your dataset containing recipes
df = pd.read_csv("/content/drive/MyDrive/total1.csv")

# Preprocessing for BM25
df['cleaned_ingredients'] = df['Ingredients'].apply(lambda x: x.lower().split(','))
tokenized_ingredients = [doc for doc in df['cleaned_ingredients']]
bm25 = BM25Okapi(tokenized_ingredients)

def search_recipes_by_ingredients(ingredients):
    query = ingredients.lower().split(',')
    scores = bm25.get_scores(query)
    top_indexes = np.argsort(scores)[::-1][:5]  # Adjust number of results as needed
    top_recipes = df.iloc[top_indexes]
    return top_recipes

config = {'max_new_tokens': 1000, 'repetition_penalty': 1.1,
          'temperature': 0.1, 'stream': False}
# Load and configure the Quantized Llama model
model_id = "TheBloke/Llama-2-13B-chat-GGML"
llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    model_type="llama",
    gpu_layers=30,
    context_length = 6000,
    **config
)

llm.config.max_new_tokens = 4096
llm.config.context_length = 8192

def generate_recipe(ingredients):
    search_results = search_recipes_by_ingredients(ingredients)
    print(search_results)
    top_recipe_titles = search_results['Recipe Title'].tolist()

    # Prepare the prompt for the generator model
    prompt = f"Generate a recipe using the following ingredients: {ingredients}. Inspired by: {', '.join(top_recipe_titles)}"

    # Generate recipe with Llama model
    tokens = llm.tokenize(prompt)
    generated_text = ''.join([llm.detokenize(token) for token in llm.generate(tokens)])

    return generated_text

# Example usage
ingredients = "flour, sugar, eggs, cream cheese"
recipe = generate_recipe(ingredients)
print(recipe)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

       Unnamed: 0                         Recipe Title  \
13879       13879              Citrus-and-Dill Gravlax   
45787       45787                        Keto brownies   
15259       15259  Zucchini Stuffed with Feta and Dill   
15267       15267                Japanese Cold Noodles   
15266       15266     Braised Shrimp and Creamy Endive   

                                             Ingredients  \
13879  ['12 servings', '1 bunch dill, plus sprigs for...   
45787  ['120g nut butter (we used almond butter)', '1...   
15259  ['Serves 4', 'four 1/2-pound zucchini, scrubbe...   
15267  ['Makes 6 servings', 'For dipping sauce', '2 c...   
15266  ['Makes 2 servings', '1 lb Belgian endives (4 ...   

                                                   Steps  \
13879  ['Step 1', 'Place dill bunch in a food process...   
45787  [{'@type': 'HowToStep', 'text': 'Heat the oven...   
15259  ['Step 1', 'Trim and discard the stem ends fro...   
15267  ['Make dipping sauce:', 'Step 1', 'Heat all